In [1]:
!pip install -U "tensorflow==2.8.*"
!pip install -U "tensorflow-text==2.8.*"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 497.9 MB 4.8 kB/s 
     |████████████████████████████████| 1.4 MB 46.8 MB/s 
     |████████████████████████████████| 462 kB 72.4 MB/s 
     |████████████████████████████████| 5.8 MB 64.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.2
    Uninstalling tensorflow-2.9.2:
      Successfully u

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
import tensorflow_hub as hub
import tensorflow_text as text

In [18]:
df_x_train = pd.read_csv("X_train.csv")
print(df_x_train)
df_y_train = pd.read_csv("y_train.csv")
print(df_y_train)

        id              keyword            location  \
0     2714              crushed         Chicago, IL   
1     7361           obliterate         Cymru araul   
2     1290               bloody             Chicago   
3     1932  burning%20buildings                 NaN   
4     4041             disaster         Calgary, AB   
...    ...                  ...                 ...   
6085  7532          oil%20spill    Street of Dallas   
6086  1821               burned  toronto Û¢ dallas   
6087  9504            terrorist                 NaN   
6088  2797               curfew            IM LOST    
6089  3667              destroy                 NaN   

                                           cleaned_text  
0       jimmyfallon crush squirrel bone mortar pestl...  
1      mccainenl think spectacular look stonewal rio...  
2     can t bloodi wait   soni set date stephen king...  
3     protest ralli stone mountain  atleast they r b...  
4      rbcinsur quot websit   disaster  tri 3 bro

In [7]:
x_column = df_x_train["cleaned_text"]
y_column = df_y_train["target"]
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
bert_preprocess = hub.KerasLayer(preprocess_url)
bert_encoder = hub.KerasLayer(encoder_url)

In [11]:
import tensorflow as tf

In [12]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='dataset_input')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout1")(outputs['pooled_output'])
l = tf.keras.layers.Dense(100,  activation='relu', name='output1')(l)
l = tf.keras.layers.Dropout(0.2, name="dropout2")(l)
l = tf.keras.layers.Dense(50, activation='relu', name='output2')(l)
l = tf.keras.layers.Dropout(0.2, name="dropout3")(l)
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 dataset_input (InputLayer)     [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['dataset_input[0][0]']          
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [15]:
model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(x_column, y_column, epochs=10)

Epoch 1/10
191/191 [==============================] - 66s 333ms/step - loss: 0.5839 - accuracy: 0.7094
Epoch 2/10
191/191 [==============================] - 64s 334ms/step - loss: 0.5680 - accuracy: 0.7246
Epoch 3/10
191/191 [==============================] - 63s 327ms/step - loss: 0.5602 - accuracy: 0.7271
Epoch 4/10
191/191 [==============================] - 63s 328ms/step - loss: 0.5542 - accuracy: 0.7358
Epoch 5/10
191/191 [==============================] - 62s 327ms/step - loss: 0.5490 - accuracy: 0.7379
Epoch 6/10
191/191 [==============================] - 63s 329ms/step - loss: 0.5508 - accuracy: 0.7369
Epoch 7/10
191/191 [==============================] - 64s 335ms/step - loss: 0.5455 - accuracy: 0.7351
Epoch 8/10
191/191 [==============================] - 63s 332ms/step - loss: 0.5413 - accuracy: 0.7433
Epoch 9/10
191/191 [==============================] - 63s 328ms/step - loss: 0.5417 - accuracy: 0.7402
Epoch 10/10
191/191 [==============================] - 63s 328ms/step - l

In [19]:
df_x_val = pd.read_csv("X_val.csv")
print(df_x_val)
df_y_val = pd.read_csv("y_val.csv")
print(df_y_val)

        id         keyword                       location  \
0     6959        massacre                       Colorado   
1     6453         injured                      Worldwide   
2     5764  forest%20fires                            NaN   
3     3841        detonate                  Morioh, Japan   
4     2341        collapse                         Europe   
...    ...             ...                            ...   
1518  7581        outbreak                    FCT, Abuja    
1519  4790       evacuated                     Manchester   
1520  1382      body%20bag  sitting on Eddie Vedders lap,   
1521  4068       displaced       Pedophile hunting ground   
1522  1105       blew%20up                            NaN   

                                           cleaned_text  
0     colorado movi massacr trial juror reach verdic...  
1     top stori   googl 4 dead dozen injur gaza blas...  
2     group florida forest servic firefight could de...  
3       tinyjecht anoth stand user 

In [22]:
x_val = df_x_val["cleaned_text"]
print(x_val)
y_val = df_y_val["target"]
print(y_val)

0       colorado movi massacr trial juror reach verdic...
1       top stori   googl 4 dead dozen injur gaza blas...
2       group florida forest servic firefight could de...
3         tinyjecht anoth stand user  deton killer queen 
4       greece  tax revenu collaps debt crisi continu ...
                              ...                        
1518    8th person die ny legionnaires  diseas outbrea...
1519    trafford centr film fan angri odeon cinema eva...
1520     questergirl ditto had  way feel drank vodka i...
1521     potu strategicpati strategi genocide  refugee...
1522    queen gambit went well anakin blew droid contr...
Name: cleaned_text, Length: 1523, dtype: object
0       1
1       1
2       1
3       0
4       1
       ..
1518    1
1519    1
1520    0
1521    1
1522    0
Name: target, Length: 1523, dtype: int64


In [23]:
model.evaluate(x_val,y_val)

48/48 [==============================] - 16s 323ms/step - loss: 0.5203 - accuracy: 0.7538


[0.5202717781066895, 0.753775417804718]